In [ ]:
!pip install tensorflow-gpu

In [ ]:
!rm -r DialogueBot
!git clone https://github.com/ressay/DialogueBot.git

In [ ]:
!pip install rdflib

In [ ]:
!pip install -r DialogueBot/requirements.txt

In [ ]:
!mkdir my_weights

In [1]:
import sys
sys.path.insert(0, ".")
from DialogueManager.FileBrowserDM.agent import AgentFB
from DialogueManager.FileBrowserDM.user_simulator import UserSimulatorFB
from DialogueManager.FileBrowserDM.file_tree_sim import FileTreeSimulator
from DialogueManager.FileBrowserDM.nlg import Nlg_system
from DialogueManager.error_model_controller import ErrorModelController
import Ontologies.onto_fbrowser as fbrowser
import argparse, json

Using TensorFlow backend.


In [2]:
CONSTANTS_FILE_PATH = 'DialogueManager/FileBrowserDM/constants.json'
constants_file = CONSTANTS_FILE_PATH

consts = {
  "run": {
    "usersim": True,
    "warmup_mem": 1000,
    "num_ep_run": 40000,
    "train_freq": 100,
    "max_round_num": 40,
    "success_rate_threshold": 0.3
  },
  "agent": {
    "save_weights_file_path": "my_weights/m.h5",
    "load_weights_file_path": 0,
    "vanilla": True,
    "learning_rate": 1e-3,
    "batch_size": 128,
    "dqn_hidden_size": 80,
    "epsilon_init": 0.5,
    "gamma": 0.7,
    "max_mem_size": 50000,
    "agent_actions": ["Create_file","Delete_file","Change_directory",
      "inform","ask","request"]
  },
  "emc": {
    "slot_error_mode": 0,
    "slot_error_prob": 0.05,
    "intent_error_prob": 0.0
  }
}
with open(constants_file, 'w') as f:
  json.dump(consts, f)

In [3]:
CONSTANTS_FILE_PATH = 'DialogueManager/FileBrowserDM/constants.json'
constants_file = CONSTANTS_FILE_PATH

with open(constants_file) as f:
  constants = json.load(f)

# Load run constants
run_dict = constants['run']
USE_USERSIM = run_dict['usersim']
WARMUP_MEM = run_dict['warmup_mem']
NUM_EP_TRAIN = run_dict['num_ep_run']
TRAIN_FREQ = run_dict['train_freq']
MAX_ROUND_NUM = run_dict['max_round_num']
SUCCESS_RATE_THRESHOLD = run_dict['success_rate_threshold']
compress = True
train_batch = True
use_encoder = False
one_hot = True

# Init. Objects
user = UserSimulatorFB(constants,fbrowser.graph)

dqn_agent = AgentFB(50, constants,train_batch, use_encoder, compress,one_hot)
error_infuser = ErrorModelController(constants)
all_actions = []
all_trees = []

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [4]:
def rm_keys(action):
    a = {}
    for key in ['intent', 'file_name', 'slot', 'new_directory', 'path', 'origin', 'dest']:
        if key in action:
            a[key] = action[key]
    return a
            

def run_round(user):
    # 1) Agent takes action given state tracker's representation of dialogue (state)
#     print('running round')
#     user.state['current_file_tree'].print_tree()
#     dqn_agent.state_tracker.get_data()['current_tree_sim'].print_tree()
    state = dqn_agent.get_state()
    
    try:
        agent_action_index, agent_action = dqn_agent.step()
        user_action, reward, done, success = user.step(agent_action)
        error_infuser.infuse_error(user_action)
        dqn_agent.update_state_user_action(user_action)
    except Exception as e:
        user.state['current_file_tree'].print_tree()
        dqn_agent.state_tracker.get_data()['current_tree_sim'].print_tree()
        for acts,trees in zip(all_actions,all_trees):
            u, a = acts
            u = rm_keys(u)
            a = rm_keys(a)
            print(u)
            print(a)
            t1,t2 = trees
            t1.print_tree()
            t2.print_tree()
        raise e
    

        
    # if not done:
        # 4) Infuse error into semantic frame level of user action
        # emc.infuse_error(user_action)
    # 5) Update state tracker with user action
    
    
    all_actions.append((agent_action, user_action))
    all_trees.append((user.state['current_file_tree'].copy(),
                    dqn_agent.state_tracker.get_data()['current_tree_sim'].copy()))
    tuser1, tagent2 = all_trees[-1]
    f, t = tuser1.tree_similarity(tagent2)
    if f != t and user_action['intent'] != 'end':
        print('not similar! ', f, t)
        print('prev:')
        tt1, tt2 = all_trees[-2]
        tt1.print_tree()
        tt2.print_tree()
        print(tt1.tree_map)
        print(tt2.tree_map)
        a, u = all_actions[-1]
        u = rm_keys(u)
        a = rm_keys(a)
        print(a)
        print(u)
        tuser1.print_tree()
        tagent2.print_tree()
        print(tuser1.tree_map)
        print(tagent2.tree_map)
        print('all_actions:', len(all_actions))
        for a, u in all_actions:
            u = rm_keys(u)
            a = rm_keys(a)
            print(a)
            print(u)
    # state_tracker.update_state_user(user_action)
    # 6) Get next state and add experience
    next_state = dqn_agent.get_state()
    # next_state = state_tracker.get_state(done)
    dqn_agent.add_experience(state, agent_action_index, reward, next_state, done)

    return reward, done, success


def train_run():
    """
    Runs the loop that trains the agent.
    Trains the agent on the goal-oriented chatbot task. Training of the agent's neural network occurs every episode that
    TRAIN_FREQ is a multiple of. Terminates when the episode reaches NUM_EP_TRAIN.
    """
    print('Training Started...')
    
    success_rate_best = 0.95
    episode = 0
    avg_tree_size_succeeded = 0.0
    tree_sizes = []
    ftree_sizes = []
    period_reward_total = 0
    period_reward_success = 0
    period_success_total = 0      
    while episode < NUM_EP_TRAIN:
        user = episode_reset()
        episode += 1
        # print('running episode:',episode)
        done = False
        # state = state_tracker.get_state()
        r_sum = 0
        while not done:
            reward, done, success = run_round(user)
            period_reward_total += reward
            r_sum += reward
              

        # print('success is: ',success)
        period_success_total += success
        period_reward_success += r_sum*success
        rsize = user.goal['goal_tree'].r_size()
        if success == 1:
          tree_sizes.append(rsize)
        else:
          ftree_sizes.append(rsize)

        # Train
        if episode % TRAIN_FREQ == 0:

            # Check success rate
            success_rate = period_success_total / TRAIN_FREQ
            avg_reward = period_reward_total / TRAIN_FREQ
            avg_success_reward = period_reward_success / max((period_success_total,1))
            if not len(tree_sizes): tree_sizes = [1]
            if not len(ftree_sizes): ftree_sizes = [1]
            print('training after getting success_rate:', success_rate, " and avg_reward: ",avg_reward, " avg success reward ", avg_success_reward,
                  " max tree size: ",max(tree_sizes)," avg size: ",float(sum(tree_sizes))/max((1,len(tree_sizes))),
                  " avg failure size: ", float(sum(ftree_sizes))/max((1,len(ftree_sizes))), " min failure size: ",min(ftree_sizes))
            
            # Update current best success rate
            
            # Flush
#             if success_rate > success_rate_best and success_rate >= SUCCESS_RATE_THRESHOLD:
#                 dqn_agent.empty_memory()
#                 period_reward_total = 0
#                 period_success_total = 0
#                 period_reward_success = 0
#                 avg_tree_size_succeeded = 0.0
#                 tree_sizes = []
#                 success_rate_best = success_rate
#                 dqn_agent.save_weights()
#                 continue
#             period_success_total = 0
#             period_reward_total = 0
            if success_rate > success_rate_best:
                # print('Episode: {} NEW BEST SUCCESS RATE: {} Avg Reward: {}' .format(episode, success_rate, avg_reward))
                success_rate_best = success_rate
                dqn_agent.save_weights()
#                 uploaded = drive.CreateFile({'title': 'm_beh.h5'})
#                 uploaded.SetContentFile('my_weights/m_beh.h5')
#                 uploaded.Upload()
#                 print('Uploaded file with ID {}'.format(uploaded.get('id')))
          
            period_reward_total = 0
            period_success_total = 0
            period_reward_success = 0
            avg_tree_size_succeeded = 0.0
            tree_sizes = []
            ftree_sizes = []
            # Copy
            dqn_agent.copy()
            # Train
            dqn_agent.train()
            
    print('...Training Ended')


def episode_reset():
    """
    Resets the episode/conversation in the warmup and training loops.
    Called in warmup and train to reset the state tracker, user and agent. Also gets the initial user action.
    """
#     user = UserSimulatorFB(constants,fbrowser.graph)
    all_actions.clear()
    all_trees.clear()
    tree_sim = FileTreeSimulator()
    data = {'current_tree_sim': tree_sim, 'tree_sim': tree_sim}
    user_action = user.reset(data)
    dqn_agent.reset(user_action,data)
    all_trees.append((user.state['current_file_tree'].copy(),
                          dqn_agent.state_tracker.get_data()['current_tree_sim'].copy()))
    return user

In [ ]:
dqn_agent.get_state_output = dqn_agent._build_state_model(dqn_agent.beh_model)
dqn_agent.get_state_and_action = dqn_agent._built_state_action_model(dqn_agent.beh_model)

In [ ]:
dqn_agent.eps = 0

In [5]:
train_run()

Training Started...
training after getting success_rate: 0.02  and avg_reward:  -6.695  avg success reward  -2.0  max tree size:  5  avg size:  3.5  avg failure size:  6.244897959183674  min failure size:  1
Epoch 1/1
178/178 [==============================] - 15s 87ms/step - loss: 0.3528
finished fitting on  2508  samples and avg triplet number:  54.16945773524721
out candidates
out dest
out dest
out dest
out dest
training after getting success_rate: 0.41  and avg_reward:  -0.525  avg success reward  2.7439024390243905  max tree size:  9  avg size:  4.634146341463414  avg failure size:  8.0  min failure size:  2
Epoch 1/1
304/304 [==============================] - 42s 139ms/step - loss: 0.6277
finished fitting on  5200  samples and avg triplet number:  65.83096153846154
training after getting success_rate: 0.22  and avg_reward:  -2.8  avg success reward  2.590909090909091  max tree size:  7  avg size:  4.0  avg failure size:  6.961538461538462  min failure size:  2
Epoch 1/1
 36/328 [

KeyboardInterrupt: 

In [ ]:
!pip install -U -q PyDrive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': 'm_beh.h5'})
uploaded.SetContentFile('my_weights/m_beh.h5')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

In [ ]:
id='18eWGaUMTpqrnjSojPEUZwJp0Vq_UrgAS'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('my_weights/m_beh.h5')

In [ ]:
def action_str(action):
    newa = {}
    for k in action:
        if k not in ['file_node','action_node']:
            newa[k] = action[k]
    return newa

In [ ]:
nlg_sys = Nlg_system()
def simulate():
    tree_sim = FileTreeSimulator()
    data = {'current_tree_sim': tree_sim, 'tree_sim': tree_sim}
    done = False
    user_action = user.reset(data)
    print('user goal:')
    tree_sim.print_tree()
    user.goal['goal_tree'].print_tree()
    print('user: ', user_action)
    
    dqn_agent.reset(user_action,data)
    while not done:
        agent_action_index, agent_action = dqn_agent.step()
        print('agent: ',nlg_sys.get_sentence(agent_action))
        user_action, reward, done, success = user.step(agent_action)
        print('user: ', user_action)
        dqn_agent.update_state_user_action(user_action)

In [ ]:
dqn_agent.eps = 0

In [ ]:
simulate()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x1 = [1,2,3] 
y1 = [2,2,1] 
# plotting the line 1 points  
plt.plot(x1, y1, label = "line 1") 
  
# line 2 points 
x2 = [1,2,3] 
y2 = [4,3,3] 
# plotting the line 2 points  
plt.plot(x2, y2, label = "line 2") 
  
# naming the x axis 
plt.xlabel('x - axis') 
# naming the y axis 
plt.ylabel('y - axis') 
# giving a title to my graph 
plt.title('Two lines on same graph!') 
  
# show a legend on the plot 
plt.legend() 
  
# function to show the plot 
# plt.show() 
plt.savefig('foo.png', bbox_inches='tight')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
episodes = [1,2,3]
success_rate = [0.1,0.4,0.6]
avg_reward = [1,4,10]
data = {
    'episodes': episodes,
    'success_rate': success_rate,
    'avg_reward': avg_reward
}
newpd = pd.DataFrame(data=data)
newpd.to_csv('file.csv')